# Proyecto final Machine Learning

Jorge Ruizvisfocri

Emilio Martinez

# Descripción del proyecto

El siguiente proyecto busca clasificar imágenes de Carcinomas Ductales Invasivos (IDC en inglés) extraidas de muestras de pacientes con cancer de mama.

La base de datos fue tomada de https://www.kaggle.com/datasets/paultimothymooney/breast-histopathology-images

Se propone utilizar una red neuronal convolucional para resolver el problema de clasificación.

# Paqueterías de trabajo

In [1]:
## Paquetes de ciencias de datos
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import math
import random

## Paquetes de lectura de imagenes
from pathlib import Path
import os
import glob

## Paquetes de imágenes
from skimage.io import imread
import cv2

# Datos de trabajo

In [2]:
# Directorio de imágenes
data_dir = Path("D://bases de datos//proyecto_ml_final//imagenes") ## Directorio de datos

Dado que todos los pacientes tienen muestras con tumores malignos y benignos, la aleatorización puede realizarse a nivel de imagen o a nivel de paciente.

Bajo el supuesto de que podría existir correlación de algún tipo entre las imágenes pertenecientes a un mismo paciente, creemos que sería interesante aleatorizar a nivel de paciente, para introducirle a la red la información de personas que jamás ha visto.

In [3]:
### Aleatorización a nivel de paciente
#### Obtenemos la lista de pacientes
px = [f for f in data_dir.iterdir() if f.is_dir()]

In [4]:
### Obtenemos el número equivalente al porcentaje deseado
k = math.ceil( len(px) * 80 // 100)

In [18]:
### Hacemos la muestra de pacientes para entrenamiento y prueba

In [5]:
### fijamos la semilla para tener reproductibilidad
random.seed(4352)

In [6]:
train_id = random.sample(px,k)

In [7]:
test_id = set(px) - set(train_id)

In [8]:
### cargamos files de entrenamiento
train_images_files = []
for p in train_id:
    img_lst = list(p.rglob("*.png"))
    train_images_files.extend(img_lst)


In [10]:
test_images_files = []
for p in test_id:
    img_lst = list(p.rglob("*.png"))
    test_images_files.extend(img_lst)

In [13]:
### Revisamos que las direcciones de las imágenes estén bien cargadas
print(
    f"Número de imagenes\n"
    "-----------------\n"
    f"Total: {len(train_images_files) + len(test_images_files) }\n"  # 277,524 tiles
    "-----------------\n"
    f"Entrenamiento: {len(train_images_files) }\n"
    "-----------------\n"
    f"Prueba: {len(test_images_files) }\n"
)

Número de imagenes
-----------------
Total: 277524
-----------------
Entrenamiento: 220389
-----------------
Prueba: 57135



## Preparamos las etiquetas

In [45]:
## y_entrenamiento
### Preparamos las etiquetas
y_train = []
for name in train_images_files:
    etiqueta = str(name)[-5]
    etiqueta_num = int(etiqueta)
    y_train.append(etiqueta_num)

In [46]:
y_test = []
for name in test_images_files:
    etiqueta = str(name)[-5]
    etiqueta_num = int(etiqueta)
    y_test.append(etiqueta_num)

In [18]:
len(y_test)

57135

In [20]:
### Revisamos números de positivos en conjuntos
print(
    f"Positivos\n"
    "-----------------\n"
    f"Total: {sum(y_train) + sum(y_test)}\n"  # 277,524 tiles
    "-----------------\n"
    f"Entrenamiento: {sum(y_train) }\n"
    "-----------------\n"
    f"Prueba: {sum(y_test) }\n"
)

Positivos
-----------------
Total: 78786.0
-----------------
Entrenamiento: 64466.0
-----------------
Prueba: 14320.0



## Preparamos las imágenes

In [21]:
### Cargamos de imágenes de entrenamiento en una lista
dataset_img_train = list()
for img in tqdm(train_images_files):
    image = Image.open(img)
    image=image.resize((50,50))
    numpydata = np.asarray(image)
    dataset_img_train.append(numpydata)
### Convertimos la lista en tensor
dataset_img_train_array = np.asarray(dataset_img_train,dtype=object)

In [ ]:
dataset_img_train_array.shape ## Debe darnos un vector de tamaño (n,a,b,c)

In [22]:
### Aplanamos las imágenes
data_set_img_train_plano = dataset_img_train_array.astype('float32') / 255

In [23]:
## Revisamos que tengan la misma longitud para el conjunto de entrenamiento
leny_train = len(y_train)
lenset_train = len(data_set_img_train_plano)

if leny_train == lenset_train:
    print("El tamaño del vector de resultados y de las imágenes es el mismo en el conjunto de entrenamiento")
else:
    print("El tamaño del vector de resultados y las imágenes no coincide en el conjunto de entrenamiento")

El tamaño del vector de resultados y de las imágenes es el mismo en el conjunto de entrenamiento


In [24]:
### Cargamos imágenes de prueba en una lista
dataset_img_test = list()
for img in tqdm(test_images_files):
    image = Image.open(img)
    image=image.resize((50,50))
    numpydata = np.asarray(image)
    dataset_img_test.append(numpydata)
### Convertimos la lista en tensor
dataset_img_test_array = np.asarray(dataset_img_test,dtype=object)

In [ ]:
dataset_img_test_array.shape ## Debe darnos un vector de tamaño (n,a,b,c)

In [25]:
### Aplanamos las imágenes
data_set_img_test_plano = dataset_img_test_array.astype('float32') / 255

In [27]:
## Revisamos que tengan la misma longitud para el conjunto de prueba
leny_test = len(y_test)
lenset_test = len(data_set_img_test_plano)

if leny_test == lenset_test:
    print("El tamaño del vector de resultados y de las imágenes es el mismo en el conjunto de prueba")
else:
    print("El tamaño del vector de resultados y las imágenes no coincide en el conjunto de prueba")

El tamaño del vector de resultados y de las imágenes es el mismo en el conjunto de prueba


# Modelo

In [88]:
### Creamos conjuntos de validación y entrenamiento
X_val = np.asarray(data_set_img_test_plano[47135:],dtype=object)
y_val = np.asarray(y_train[47135:],dtype=object)
X_train_2 = np.asarray(data_set_img_test_plano[:47135],dtype=object)
y_train_2 = np.asarray(y_train[:47135],dtype=object)

In [77]:
### Usamos modelo de la tarea 11
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same', input_shape=(50, 32, 3)))
model.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
#model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
#model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [78]:
model.compile(Adam(learning_rate=.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [79]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 50, 50, 32)        896       
                                                                 
 conv2d_21 (Conv2D)          (None, 50, 50, 32)        9248      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 25, 25, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 25, 25, 64)        18496     
                                                                 
 conv2d_23 (Conv2D)          (None, 25, 25, 64)        36928     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 12, 12, 64)       0         
 g2D)                                                 

In [89]:
## Entrenamos el modelo
history = model.fit(X_train_2, y_train_2, batch_size=256, epochs=2, validation_data=(X_val, y_val), shuffle=True)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [90]:
Xt

numpy.ndarray

C:\Users\jorge\AppData\Local\Temp\ipykernel_6756\3453173665.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array(X_train_2)


In [92]:
X_train_2[5]

array([[[0.6901961 , 0.5411765 , 0.8666667 ],
        [0.7019608 , 0.5686275 , 0.7411765 ],
        [0.8117647 , 0.7294118 , 0.8901961 ],
        ...,
        [0.6627451 , 0.4862745 , 0.88235295],
        [0.6509804 , 0.49803922, 0.8784314 ],
        [0.58431375, 0.39215687, 0.85490197]],

       [[0.7647059 , 0.654902  , 0.8862745 ],
        [0.6392157 , 0.4509804 , 0.69803923],
        [0.827451  , 0.7372549 , 0.9137255 ],
        ...,
        [0.5882353 , 0.4       , 0.8509804 ],
        [0.6156863 , 0.44313726, 0.81960785],
        [0.5803922 , 0.3882353 , 0.81960785]],

       [[0.7490196 , 0.627451  , 0.8745098 ],
        [0.5647059 , 0.3882353 , 0.7607843 ],
        [0.62352943, 0.4627451 , 0.81960785],
        ...,
        [0.59607846, 0.39215687, 0.85490197],
        [0.6666667 , 0.5019608 , 0.8745098 ],
        [0.5137255 , 0.30588236, 0.7607843 ]],

       ...,

       [[0.5529412 , 0.34117648, 0.5882353 ],
        [0.81960785, 0.7882353 , 0.8666667 ],
        [0.5921569 , 0